In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = [16,9]

### Reading in Release CSV

In [2]:
release_header = ['id', 'target', 'date', 'date_offset', 'name', 'comment', 'author']
# Path to release.csv, and whether to select uncompressed or ocmpressed version
# release_path = 'D:/data/open_source/release.csv'
release_path = 'C:\\Users\\Faugh\\Desktop\\DataScienceData\\popular-3k-python\\release.csv'

In [3]:
def hex_str(str):
    return bytes.fromhex((str).replace('\\x','')).decode('utf-8')

def date_str(str):
    return pd.to_datetime(str, format='%Y-%m-%dT%H:%M:%S.000Z')

In [4]:
release_df = pd.read_csv(release_path, names=release_header, converters={'date':date_str,'comment':hex_str})
release_df = release_df[['id','date','name','comment','author']]

In [5]:
release_df.head()

,id,date,name,comment,author
0,\xae671a0067dbeabbc3cb546705edc1f81f71a193,2015-02-03 21:58:44,\x72656c656173652d323031352d30322d3034,"Release for February 4th, 2015\n",91949
1,\x33e2c27d1ec224a54ed7ca4a6e09c352e92a227d,2013-07-05 04:58:14,\x76302e32392e31,tag version 0.29.1\n,140982
2,\x1f9bcd823807f41afaab6b74b34473531ca7eb30,2013-03-29 13:02:20,\x76332e302e30,What's new in Tornado 3.0\n===================...,207066
3,\x3206fb28a040494bac6973310e7f21f031989da6,2010-12-09 04:14:19,\x6275696c642d343639,Windows build SickBeard-win32-alpha-build469\n,875380
4,\x652fc27cf9fe9262d2c941d6385043efa41016da,2011-11-08 01:22:48,\x312e312e36,Fabric 1.1.6\n,63770


### Reading in NVD CSV's

In [6]:
import glob

path = '../data/NVD-to-CSV-Descriptions/'
allcves = glob.glob(path+'*.csv')
l = []

for filename in allcves:
    df = pd.read_csv(filename)
    l.append(df)
df_NVD_hash = pd.concat(l, ignore_index=True)
#New cell
df_NVD_hash

,conCpe,conOper,conVer,conVul,cve.CVE_data_meta.ID,desLang,desValue,proLang,proValue,refName,refSource,refTags,refUrl
0,NaN,NaN,NaN,NaN,CVE-1999-0001,en,ip_input.c in BSD-derived TCP/IP implementatio...,en,CWE-20,5707,OSVDB,[],http://www.osvdb.org/5707
1,NaN,NaN,NaN,NaN,CVE-1999-0002,en,Buffer overflow in NFS mountd gives root acces...,en,CWE-119,121,BID,"['Exploit', 'Patch', 'Vendor Advisory']",http://www.securityfocus.com/bid/121
2,NaN,NaN,NaN,NaN,CVE-1999-0003,en,Execute commands as root via buffer overflow i...,en,NVD-CWE-Other,122,BID,[],http://www.securityfocus.com/bid/122
3,NaN,NaN,NaN,NaN,CVE-1999-0004,en,"MIME buffer overflow in email clients, e.g. So...",en,NVD-CWE-Other,MS98-008,MS,[],https://docs.microsoft.com/en-us/security-upda...
4,NaN,NaN,NaN,NaN,CVE-1999-0005,en,Arbitrary command execution via IMAP buffer ov...,en,NVD-CWE-Other,130,BID,[],http://www.securityfocus.com/bid/130
5,NaN,NaN,NaN,NaN,CVE-1999-0006,en,Buffer overflow in POP servers based on BSD/Qu...,en,NVD-CWE-Other,133,BID,[],http://www.securityfocus.com/bid/133
6,NaN,NaN,NaN,NaN,CVE-1999-0007,en,Information from SSL-encrypted sessions via PK...,en,NVD-CWE-Other,MS98-002,MS,[],https://docs.microsoft.com/en-us/security-upda...
7,NaN,NaN,NaN,NaN,CVE-1999-0008,en,"Buffer overflow in NIS+, in Sun's rpc.nisd pro...",en,NVD-CWE-Other,00170,SUN,[],http://sunsolve.sun.com/pub-cgi/retrieve.pl?do...
8,NaN,NaN,NaN,NaN,CVE-1999-0009,en,Inverse query buffer overflow in BIND 4.9 and ...,en,NVD-CWE-Other,HPSBUX9808-083,HP,[],http://www1.itrc.hp.com/service/cki/docDisplay...
9,NaN,NaN,NaN,NaN,CVE-1999-0010,en,Denial of Service vulnerability in BIND 8 Rele...,en,NVD-CWE-Other,HPSBUX9808-083,HP,[],http://www1.itrc.hp.com/service/cki/docDisplay...
